<a href="https://colab.research.google.com/github/lizcastillo-glitch/Proyecto-IA-Ecommerce-sentimientos-/blob/main/src/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install deep_translator
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from deep_translator import GoogleTranslator
import torch.nn.functional as F
import os

class IAEcommerceAmazonReviewDashboard:
    def __init__(self, model_path):
        """
        Carga el modelo y el tokenizador al iniciar la clase.
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Cargando modelo desde {model_path} en {self.device}...")

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
            self.model.to(self.device)
            self.model.eval() # Modo evaluación
            print("✅ Modelo cargado exitosamente.")
        except Exception as e:
            print(f"❌ Error cargando el modelo: {e}")
            self.tokenizer = None
            self.model = None

    def predict(self, text_es):
        """
        Recibe texto en español, traduce e infiere.
        """
        if not self.model or not self.tokenizer:
            return {"error": "Modelo no cargado"}

        # 1. Traducción (Híbrida)
        try:
            translator = GoogleTranslator(source='auto', target='en')
            text_en = translator.translate(text_es)
        except Exception as e:
            return {"error": f"Fallo en traducción: {str(e)}"}

        # 2. Preprocesamiento
        inputs = self.tokenizer(
            text_en,
            return_tensors="pt",
            truncation=True,
            max_length=128
        ).to(self.device)

        # 3. Inferencia
        with torch.no_grad():
            logits = self.model(**inputs).logits

        probs = F.softmax(logits, dim=-1).squeeze().cpu().numpy()
        pred_idx = probs.argmax()

        # 4. Regla de Neutros (Umbral 0.22)
        top1 = probs[pred_idx]
        top2 = sorted(probs, reverse=True)[1]

        if (top1 - top2) < 0.22:
            pred_idx = 1 # Forzar Neutro

        labels = {0: "Negativo", 1: "Neutro", 2: "Positivo"}

        return {
            "sentimiento": labels[pred_idx],
            "score_confianza": float(probs[pred_idx]),
            "traduccion_usada": text_en,
            "probabilidades": {
                "negativo": float(probs[0]),
                "neutro": float(probs[1]),
                "positivo": float(probs[2])
            }
        }

First, let's define the model path. For sentiment analysis, a common choice is `cardiffnlp/twitter-roberta-base-sentiment-latest` from Hugging Face, which is trained on Twitter data and performs well on general sentiment tasks. You can replace this with any other compatible model if you have a specific one in mind.

In [8]:
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

# Instantiate the class
sentiment_analyzer = IAEcommerceAmazonReviewDashboard(model_path)


Cargando modelo desde cardiffnlp/twitter-roberta-base-sentiment-latest en cpu...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✅ Modelo cargado exitosamente.


Now that the class is instantiated and the model is loaded, you can use the `predict` method to analyze a Spanish review. Let's try an example:

In [9]:
review_es = "Este producto es increíble, me encanta su calidad y funciona perfectamente."
prediction = sentiment_analyzer.predict(review_es)
print(prediction)

review_es_2 = "El producto llegó roto y la calidad es muy mala, estoy muy decepcionado."
prediction_2 = sentiment_analyzer.predict(review_es_2)
print(prediction_2)

review_es_3 = "El envío fue rápido, pero el producto es solo promedio, nada especial."
prediction_3 = sentiment_analyzer.predict(review_es_3)
print(prediction_3)

{'sentimiento': 'Positivo', 'score_confianza': 0.9869662523269653, 'traduccion_usada': 'This product is amazing, I love the quality and it works perfectly.', 'probabilidades': {'negativo': 0.0052098422311246395, 'neutro': 0.00782396923750639, 'positivo': 0.9869662523269653}}
{'sentimiento': 'Negativo', 'score_confianza': 0.946765124797821, 'traduccion_usada': 'The product arrived broken and the quality is very bad, I am very disappointed.', 'probabilidades': {'negativo': 0.946765124797821, 'neutro': 0.04686260595917702, 'positivo': 0.006372309289872646}}
{'sentimiento': 'Negativo', 'score_confianza': 0.8450381755828857, 'traduccion_usada': 'Shipping was fast, but the product is just average, nothing special.', 'probabilidades': {'negativo': 0.8450381755828857, 'neutro': 0.13776527345180511, 'positivo': 0.01719646155834198}}
